In [5]:
import requests
import psycopg2
import pandas as pd

from bs4 import BeautifulSoup


MENU_URI = "https://mymall.se/media/mymall/megamenu/9826648.html"

data = []
uri_response = requests.get(url=MENU_URI).content
bs_obj = BeautifulSoup(markup=uri_response, parser='html.parser', features='lxml')

unordered_li = bs_obj.select('body > li')

for un_lis in unordered_li:
    # get the mega category info first
    mega_cat_name = un_lis.contents[1].text.strip()
    mega_cat_uri = un_lis.contents[1].find('a', href=True).get('href')

    # get main categories from the mega category
    main_cat = un_lis.select('div > ul > li')
    for main_cats in main_cat:
        try:
            content = main_cats.contents[1]
        except IndexError:
            content = main_cats.contents[0]
        main_cat_name = content.text.strip()

        try:
            main_cat_uri = content.contents[1].get('href')
        except IndexError:
            main_cat_uri = content.get('href')

        # get sub-categories from main category
        sub_cat = main_cats.select('div > ul > li')
        for sub_cats in sub_cat:
            sub_cat_name = sub_cats.text
            sub_cat_uri = sub_cats.contents[0].get('href')

            data.append({
                'mega_category': mega_cat_name,
                'main_category': main_cat_name,
                'sub_category': sub_cat_name,
                'mega_category_url': mega_cat_uri,
                'main_category_url': main_cat_uri,
                'sub_category_url': sub_cat_uri
            })

    # add data into panda dataframe
    df = pd.DataFrame(data=data, columns=['mega_category', 'main_category', 'sub_category',
                                          'mega_category_url', 'main_category_url', 'sub_category_url'])


In [6]:
df

,mega_category,main_category,sub_category,mega_category_url,main_category_url,sub_category_url
0,Inredning & Möbler,Barnrummet,Belysning,https://mymall.se/inredning-mobler,https://mymall.se/inredning-mobler/barnrummet,https://mymall.se/inredning-mobler/barnrummet/...
1,Inredning & Möbler,Barnrummet,Förvaring,https://mymall.se/inredning-mobler,https://mymall.se/inredning-mobler/barnrummet,https://mymall.se/inredning-mobler/barnrummet/...
2,Inredning & Möbler,Barnrummet,Hemsäkerhet,https://mymall.se/inredning-mobler,https://mymall.se/inredning-mobler/barnrummet,https://mymall.se/inredning-mobler/barnrummet/...
3,Inredning & Möbler,Barnrummet,Inredningsdetaljer,https://mymall.se/inredning-mobler,https://mymall.se/inredning-mobler/barnrummet,https://mymall.se/inredning-mobler/barnrummet/...
4,Inredning & Möbler,Barnrummet,Junior- & barnsängar,https://mymall.se/inredning-mobler,https://mymall.se/inredning-mobler/barnrummet,https://mymall.se/inredning-mobler/barnrummet/...
...,...,...,...,...,...,...
697,Hemelektronik,Smarta hemmet,Övervakningskameror,https://mymall.se/hemelektronik,https://mymall.se/hemelektronik/smarta-hemmet,https://mymall.se/hemelektronik/smarta-hemmet/...
698,Hemelektronik,Vitvaror,Bänkspisar & Kokplattor,https://mymall.se/hemelektronik,https://mymall.se/hemelektronik/vitvaror,https://mymall.se/hemelektronik/vitvaror/kokpl...
699,Hemelektronik,Vitvaror,Kyl & Frys,https://mymall.se/hemelektronik,https://mymall.se/hemelektronik/vitvaror,https://mymall.se/hemelektronik/vitvaror/kyl-frys
700,Hemelektronik,Vitvaror,Mikrovågsugnar,https://mymall.se/hemelektronik,https://mymall.se/hemelektronik/vitvaror,https://mymall.se/hemelektronik/vitvaror/mikro...
